<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using SingleStore with OpenAI's Swarm</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!pip install git+https://github.com/openai/swarm.git --quiet
!pip install langchain --quiet
!pip install langchain-community --quiet
!pip install langchain-openai --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install unstructured==0.10.14 --quiet

In [8]:
import nltk
import shutil

from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import SingleStoreDB
from langchain_openai import OpenAIEmbeddings
from singlestoredb.management import get_secret
from swarm import Swarm, Agent

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [9]:
from sqlalchemy import *

conn = create_engine(connection_url)

In [10]:
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger_eng")

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [11]:
loader = OnlinePDFLoader("https://www.investni.com/sites/default/files/2021-02/NI-fintech-document.pdf")

data = loader.load()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [12]:
print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 39674 characters in your document


In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 20
)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 23 pages


In [14]:
os.environ["OPENAI_API_KEY"] = get_secret("OPENAI_API_KEY")

In [15]:
%%sql
DROP TABLE IF EXISTS fintech_docs;

++
||
++
++

In [16]:
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "fintech_docs"
)

In [17]:
# Define the search function
def search_s2(query_text: str) -> str:
    # Perform a similarity search and return the first result's content
    docs = docsearch.similarity_search(query_text)
    return docs[0].page_content

# Define a function to transfer control to Agent B
def transfer_to_agent_b():
    return agent_b

In [18]:
# Define shared context to hold information between agents
shared_context = {}

# Define Agent A to focus on blockchain context
agent_a = Agent(
    name = "Blockchain Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in blockchain technology.",
    functions = [search_s2, transfer_to_agent_b]
)

# Define Agent B to focus on investment opportunities
agent_b = Agent(
    name = "Investment Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in stock market investment."
)

In [19]:
client = Swarm()

# Run Agent A with a blockchain-specific query
response_a = client.run(
    agent = agent_a,
    messages = [{
        "role": "user",
        "content": "What are the latest developments and trends in blockchain technology in Northern Ireland?"
    }],
)

# Store Agent A's response in shared context for Agent B to expand on
shared_context["blockchain_context"] = response_a.messages[-1]["content"]

# Output the response from Agent A
print(shared_context["blockchain_context"])

Recent developments and trends in blockchain technology in Northern Ireland illustrate a vibrant and rapidly growing ecosystem, particularly within the FinTech sector. Here are the key takeaways:

1. **FinTech as a Central Focus**: Northern Ireland is emerging as a center of excellence for FinTech, attracting international firms due to its combination of world-class talent, competitive costs, and robust infrastructure. The region is becoming a global hub for FinTech investments.

2. **Strong Industry Presence**: Major companies such as Citi, Allstate, Liberty Mutual, and FD Technologies have established significant operations in Northern Ireland. Recent entrants like CME Group, Vela, and TP ICAP have also set up large delivery centers, reinforcing the region's reputation in financial services.

3. **Entrepreneurial Ecosystem**: The start-up and SME tech communities are thriving, supported by a strong entrepreneurial ecosystem, networking opportunities, and sector collaboration. This en

In [20]:
# Run Agent B with a focus on investment opportunities within the blockchain context
response_b = client.run(
    agent = agent_b,
    messages = [{
        "role": "user",
        "content": f"{shared_context['blockchain_context']} Based on these developments, what are the best investment opportunities in blockchain in Northern Ireland?"
    }],
)

# Output the final response from Agent B
print(response_b.messages[-1]["content"])

Based on the recent developments and trends in blockchain technology in Northern Ireland, several investment opportunities in the blockchain and FinTech space can be identified. Here are some potential avenues for investment:

1. **Investing in FinTech Startups**: Given the thriving entrepreneurial ecosystem, there are opportunities to invest in promising early-stage FinTech startups that are utilizing blockchain technology. Look for companies focused on payment solutions, decentralized finance (DeFi), or blockchain-based identity management, as these areas are likely to see significant growth.

2. **Venture Capital Funds**: Consider investing in venture capital funds that are specifically focused on FinTech and blockchain technology in Northern Ireland. These funds typically have access to a pipeline of innovative companies and can provide diversification in your investment portfolio.

3. **Established Financial Institutions**: Major firms like Citi, Allstate, and CME Group are expand

## Refactor Code

In [22]:
# Define the search function
def search_s2(query_text) -> str:
    # Perform a similarity search and return the first result's content
    docs = docsearch.similarity_search(query_text)
    return docs[0].page_content

# Define a function to transfer control to Agent A
def transfer_to_agent_a():
    return agent_a

# Define a function to transfer control to Agent B
def transfer_to_agent_b():
    return agent_b

In [23]:
# Define Agent A to focus on blockchain context
agent_a = Agent(
    name = "Blockchain Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in blockchain technology.",
    functions = [search_s2, transfer_to_agent_b]
)

# Define Agent B to focus on investment opportunities
agent_b = Agent(
    name = "Investment Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in stock market investment.",
    functions = [search_s2, transfer_to_agent_a]
)

In [24]:
client = Swarm()

# Run Agent
response = client.run(
    agent = agent_a,
    messages = [{
        "role": "user",
        "content": (
            "What are the latest developments and trends in blockchain technology "
            "in Northern Ireland? Based on these developments, what are the best "
            "investment opportunities in blockchain in Northern Ireland?"
        )
    }],
)

print(response.messages[-1]["content"])

In Northern Ireland, the latest developments and trends in blockchain technology, particularly in 2023, indicate a strong growth trajectory fueled by several key factors:

1. **FinTech Hub**: Northern Ireland has emerged as a center of excellence for financial technology (FinTech). The region benefits from world-class talent, competitive operating costs, and a resilient business infrastructure. This environment is highly conducive to innovation and the adoption of new technologies, including blockchain.

2. **International Investments**: Global firms such as Citi, Allstate, Liberty Mutual, and FD Technologies have established or expanded their operations in Northern Ireland. New entrants like CME Group, Vela, and TP ICAP have also set up significant operations, indicating a thriving ecosystem for FinTech that leverages blockchain technology.

3. **Entrepreneurial Ecosystem**: The start-up and SME tech community in Northern Ireland is flourishing, supported by a vibrant networking scene

## Cleanup

In [25]:
shutil.rmtree("nltk_data")

In [26]:
%%sql
DROP TABLE IF EXISTS fintech_docs;

++
||
++
++